In [6]:
import pandas as pd
import lightgbm as lgb
import pickle
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import numpy as np

# --- Configuration ---
# IMPORTANT: Make sure these paths are correct for your local setup.
DATA_PATH = '/Users/rushilpatel/Downloads/hull-tactical-market-prediction/'
OUTPUT_PATH = '/Users/rushilpatel/Downloads/hull-tactical-market-prediction/'

TRAIN_FILE_PATH = os.path.join(DATA_PATH, 'train.csv')
MODEL_OUTPUT_PATH = os.path.join(OUTPUT_PATH, 'lgbm_model.pkl')
FEATURES_OUTPUT_PATH = os.path.join(OUTPUT_PATH, 'selected_features.txt')
DAYS_PER_YEAR = 252
ALLOCATION_SCALING_FACTOR = 7.0

# --- Custom Metric Implementation ---
# The functions below translate the competition's scoring formula into code
# that LightGBM can use for its objective and evaluation metric.

def calculate_final_score(y_true, y_pred, df_for_metric):
    """Calculates the volatility- and return-penalized Sharpe ratio."""
    
    # Ensure y_true and y_pred are numpy arrays
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)

    # Use the relevant part of the dataframe for market and risk-free returns
    market_returns = df_for_metric['forward_returns'].values
    risk_free_rate = df_for_metric['risk_free_rate'].values

    # 1. Calculate Strategy and Market Returns
    positions = np.clip(1.0 + y_pred * ALLOCATION_SCALING_FACTOR, 0, 2)
    strategy_returns = risk_free_rate * (1 - positions) + positions * market_returns
    
    strategy_excess_returns = strategy_returns - risk_free_rate
    market_excess_returns = market_returns - risk_free_rate

    # Handle potential empty arrays or all-zero returns
    if len(strategy_excess_returns) == 0 or np.all(strategy_excess_returns == 0):
        return 0.0

    # 2. Calculate Mean Returns and Volatility
    # Geometric Mean (add 1 to handle returns, subtract 1 at the end)
    geo_mean_strategy = np.expm1(np.mean(np.log1p(strategy_excess_returns)))
    geo_mean_market = np.expm1(np.mean(np.log1p(market_excess_returns)))

    # Annualized Volatility
    vol_strategy = np.std(strategy_returns, ddof=1) * np.sqrt(DAYS_PER_YEAR)
    vol_market = np.std(market_returns, ddof=1) * np.sqrt(DAYS_PER_YEAR)
    
    # Avoid division by zero
    if vol_strategy == 0:
        return 0.0

    # 3. Base Sharpe Ratio
    sharpe_ratio = (geo_mean_strategy / vol_strategy) * np.sqrt(DAYS_PER_YEAR)

    # 4. Penalty Factors
    # Volatility Penalty
    vol_penalty = 1 + max(0, (vol_strategy / vol_market) - 1.2)
    
    # Return Penalty
    annualized_return_gap = max(0, (geo_mean_market - geo_mean_strategy) * DAYS_PER_YEAR)
    return_penalty = 1 + (annualized_return_gap ** 2) / 100

    # 5. Final Score
    final_score = min(sharpe_ratio / (vol_penalty * return_penalty), 1_000_000)
    
    return final_score

def sharpe_objective(y_true, y_pred):
    """
    Custom objective function for LightGBM.
    We need to return gradient and hessian. For ranking/optimization tasks,
    a common trick is to use a simplified objective (like L2 loss) for stable gradients,
    while using the exact metric for evaluation. Here we'll return L2 gradients
    but the key is using the custom eval metric for early stopping.
    """
    grad = (y_pred - y_true)
    hess = np.ones_like(grad)
    return grad, hess

def sharpe_eval_metric(y_true, y_pred):
    """
    Custom evaluation metric for LightGBM's early stopping.
    This function will be called with the validation data.
    """
    # We need access to the corresponding validation dataframe to get market/risk-free returns
    # This is a bit of a hack: we rely on the global validation dataframe `X_val_df`
    score = calculate_final_score(y_true, y_pred, val_metric_df)
    return 'custom_sharpe', score, True # True indicates that a higher score is better


def engineer_features(df):
    """
    Creates new interaction and lag features to provide the model with more signals.
    """
    print("Engineering new features...")
    df['P1_div_V1'] = df['P1'] / (df['V1'] + 1e-6)
    df['I1_mul_E1'] = df['I1'] * df['E1']
    lags = [1, 3, 5]
    features_to_lag = ['M1', 'V1', 'P1', 'S1']
    for feature in features_to_lag:
        for lag in lags:
            df[f'{feature}_lag_{lag}'] = df[feature].shift(lag)
    return df

# --- Global dataframe for metric calculation ---
# This is needed so our eval metric can access the other columns (returns, risk-free rate)
val_metric_df = None

def train_and_evaluate_model():
    """
    A comprehensive workflow to load data, engineer features, train a LightGBM model
    optimized for a custom Sharpe ratio, and save the final artifacts.
    """
    global val_metric_df

    print("--- Starting Model Training & Evaluation Workflow ---")

    # --- 1. Load Data ---
    try:
        print(f"Loading training data from: {TRAIN_FILE_PATH}")
        df_train = pd.read_csv(TRAIN_FILE_PATH)
        print("Training data loaded successfully.")
    except FileNotFoundError:
        print(f"ERROR: Training file not found at {TRAIN_FILE_PATH}")
        return

    # --- 2. Feature Selection ---
    base_features = [
        'M1', 'M5', 'M6', 'E1', 'E5', 'E7', 'I1', 'I5', 'I7',
        'P1', 'P5', 'P6', 'V1', 'V5', 'V8', 'V9',
        'S1', 'S5', 'S8', 'S10', 'D1', 'D5', 'D8'
    ]
    target = 'market_forward_excess_returns'
    
    # --- 3. Feature Engineering ---
    df_train = engineer_features(df_train)
    
    engineered_feature_names = [col for col in df_train.columns if col not in ['date_id', target, 'market_forward_excess_returns']]
    selected_features = [f for f in engineered_feature_names if f in base_features or '_lag_' in f or '_div_' in f or '_mul_' in f]
    
    df_train.dropna(subset=[target], inplace=True)
    
    X = df_train[selected_features].ffill().bfill()
    y = df_train[target]

    # --- 4. Time-Series Split ---
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, shuffle=False)
    
    # Create the global validation dataframe for the metric function
    # It must contain the features AND the columns needed for the metric
    val_metric_df = df_train.loc[X_val.index]

    print(f"Training data shape: {X_train.shape}")
    print(f"Validation data shape: {X_val.shape}")
    
    # --- 5. Model Training with Custom Objective ---
    print("\n--- 5. Model Training with Custom Sharpe Metric ---")
    lgbm = lgb.LGBMRegressor(
        random_state=42,
        n_estimators=2000,
        learning_rate=0.01,
        num_leaves=25,
        min_child_samples=40,
        subsample=0.8,
        colsample_bytree=0.8,
        reg_alpha=0.5,
        reg_lambda=2.5,
        objective=sharpe_objective # Use a simplified objective for stability
    )

    lgbm.fit(X_train, y_train,
             eval_set=[(X_val, y_val)],
             eval_metric=sharpe_eval_metric, # Use the precise metric for evaluation
             callbacks=[lgb.early_stopping(100, verbose=True), lgb.log_evaluation(100)])

    # --- 6. Analysis & Visualization ---
    print("\n--- 6. Model Analysis ---")
    
    # Plot Feature Importance
    lgb.plot_importance(lgbm, max_num_features=25, figsize=(10, 10), importance_type='gain')
    plt.title('Top 25 Feature Importances (Gain)')
    plt.show()

    # --- 7. Final Model Training & Saving ---
    print("\n--- 7. Training Final Model on All Data ---")
    final_model = lgb.LGBMRegressor(**lgbm.get_params())
    final_model.set_params(n_estimators=lgbm.best_iteration_)
    final_model.fit(X, y)
    print("Final model training complete.")

    print(f"Saving final model to: {MODEL_OUTPUT_PATH}")
    with open(MODEL_OUTPUT_PATH, 'wb') as f:
        pickle.dump(final_model, f)
    print("Model saved successfully.")
    
    print(f"Saving feature list to: {FEATURES_OUTPUT_PATH}")
    with open(FEATURES_OUTPUT_PATH, 'w') as f:
        for feature in X.columns:
            f.write(f"{feature}\n")
    print("Feature list saved successfully.")
    print("\n--- Workflow Finished Successfully! ---")


if __name__ == '__main__':
    if not os.path.exists(OUTPUT_PATH):
        os.makedirs(OUTPUT_PATH)
    train_and_evaluate_model()



--- Starting Model Training & Evaluation Workflow ---
Loading training data from: /Users/rushilpatel/Downloads/hull-tactical-market-prediction/train.csv
Training data loaded successfully.
Engineering new features...
Training data shape: (7192, 37)
Validation data shape: (1798, 37)

--- 5. Model Training with Custom Sharpe Metric ---
[LightGBM] [Info] Using self-defined objective function
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000721 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 8429
[LightGBM] [Info] Number of data points in the train set: 7192, number of used features: 37
[LightGBM] [Info] Using self-defined objective function
Training until validation scores don't improve for 100 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive 


--- 7. Training Final Model on All Data ---
[LightGBM] [Info] Using self-defined objective function
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000369 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 8656
[LightGBM] [Info] Number of data points in the train set: 8990, number of used features: 37
[LightGBM] [Info] Using self-defined objective function
Final model training complete.
Saving final model to: /Users/rushilpatel/Downloads/hull-tactical-market-prediction/lgbm_model.pkl
Model saved successfully.
Saving feature list to: /Users/rushilpatel/Downloads/hull-tactical-market-prediction/selected_features.txt
Feature list saved successfully.

--- Workflow Finished Successfully! ---
